# Binary knapsack optimization on a qumode

## Prerequisite

Cell for Google Colab users.

In [ ]:
!pip install git+https://github.com/sympy/sympy.git
!pip install qutip
!pip install scipy

  Cloning https://github.com/sympy/sympy.git to /tmp/pip-req-build-q9_409bs
  Running command git clone --filter=blob:none --quiet https://github.com/sympy/sympy.git /tmp/pip-req-build-q9_409bs
  Resolved https://github.com/sympy/sympy.git to commit c741214e5be1cdd0ede61bc01eecf0eb3a67ea4e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sympy: filename=sympy-1.14.dev0-py3-none-any.whl size=6256771 sha256=35d24fb1979bd6ab8cae3417b22af6d662ba34dd4d8608239fea1da6305e21b1
  Stored in directory: /tmp/pip-ephem-wheel-cache-_wlddhim/wheels/77/06/c3/4de376dd4507851c07f5b7581d16e85d493a4ec0f0adbc6423
Successfully built sympy
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

Packages.

In [ ]:
import numpy as np
import qutip as qt
import sympy as sp
import scipy.optimize as sciopt

from functools import partial

In [ ]:
import matplotlib.pyplot as plt

## Ansatz

### Basics

In [ ]:
def get_cvec_np(r, theta):
    r = np.array(r)
    theta = np.array(theta)
    return r * np.exp(1j * theta)

In [ ]:
def pack_variables(beta_mag, beta_arg, theta, phi):
    Xvec = np.concatenate([
        beta_mag.ravel(),
        beta_arg.ravel(),
        theta.ravel(),
        phi.ravel()
    ])
    return Xvec


def unpack_variables(Xvec, ndepth):
    size = ndepth * 2

    beta_mag = Xvec[:size].reshape((ndepth, 2))
    beta_arg = Xvec[size:2*size].reshape((ndepth, 2))
    theta = Xvec[2*size:3*size].reshape((ndepth, 2))
    phi = Xvec[3*size:].reshape((ndepth, 2))

    return beta_mag, beta_arg, theta, phi

In [ ]:
def qproj00():
    return qt.basis(2, 0).proj()


def qproj11():
    return qt.basis(2, 1).proj()


def qproj01():
    op = np.array([[0, 1], [0, 0]])
    return qt.Qobj(op)


def qproj10():
    op = np.array([[0, 0], [1, 0]])
    return qt.Qobj(op)

### ECDs with qubit rotations

Qubit rotation with qumode echoed conditional displacement (ECD) operators ([reference](https://doi.org/10.1038/s41567-022-01776-9)) for one qubit and two qumodes

\begin{align*}
U (\vec{\beta}, \vec{\theta}, \vec{\phi})
&= ECD_2 (\beta_2) \:
\big[ R (\theta_2, \phi_2) \otimes I \otimes I \big] \:
ECD_1 (\beta_1) \:
\big[ R (\theta_1, \phi_1) \otimes I \otimes I \big]
\\
ECD_1 (\beta_1)
&= |1 \rangle \langle 0| \otimes D (\beta_1 / 2) \otimes I  
+ |0 \rangle \langle 1| \otimes D (-\beta_1 / 2) \otimes I,
\\
ECD_2 (\beta_2)
&= |1 \rangle \langle 0| \otimes I \otimes D (\beta_2 / 2)
+ |0 \rangle \langle 1| \otimes I \otimes D (-\beta_2 / 2),
\end{align*}

where
$ R (\theta, \phi)
= e^{ - i (\theta / 2) \big[ \cos(\phi) X + \sin(\phi) Y \big] } $ and
$ D (\beta) = e^{ \beta a^\dagger - \beta^* a } $.

In [ ]:
def qubit_rot(theta, phi):
    """
    R (theta, phi) = exp[ −i (theta/2) ( X cos(phi) + Y sin(phi) ) ].

    Arguments:
    theta, phi: rotation parameters
    """
    gen = ( qt.sigmax() * np.cos(phi) )
    gen += ( qt.sigmay() * np.sin(phi) )

    H = -1j * (theta / 2) * gen

    return H.expm()

In [ ]:
def ecd_op(beta, theta, phi, cind, nfocks):
    """
    ECD operator.

    Arguments:
    beta -- ECD parameter
    theta, phi -- rotation parameters
    cind -- qumode index
    nfocks -- Fock cutoffs
    """
    # Validate cind
    if cind not in (0, 1):
        raise ValueError("cind must be 0 or 1")

    # ECD
    if cind == 0:
        E2 = qt.tensor(qproj10(), qt.displace(nfocks[0], beta / 2))
        E2 += qt.tensor(qproj01(), qt.displace(nfocks[0], -beta / 2))
        E2 = qt.tensor(E2, qt.qeye(nfocks[1]))
    else:
        E2 = qt.tensor(qproj10(), qt.qeye(nfocks[0]), qt.displace(nfocks[1], beta / 2))
        E2 += qt.tensor(qproj01(), qt.qeye(nfocks[0]), qt.displace(nfocks[1], -beta / 2))

    return E2

In [ ]:
def ecd_rot_op(beta, theta, phi, nfocks):
    """
    ECD-rotation operator.

    Arguments:
    beta -- ECD parameters
    theta, phi -- rotation parameters
    nfocks -- Fock cutoffs
    """
    # Rotations
    R1 = qt.tensor(qubit_rot(theta[0], phi[0]), qt.qeye(nfocks[0]), qt.qeye(nfocks[1]))
    R2 = qt.tensor(qubit_rot(theta[1], phi[1]), qt.qeye(nfocks[0]), qt.qeye(nfocks[1]))

    # ECDs
    E1 = ecd_op(beta[0], theta[0], phi[0], 0, nfocks)
    E2 = ecd_op(beta[1], theta[1], phi[1], 1, nfocks)

    return E2 * R2 * E1 * R1

Build the ansatz matrix of depth $N_d$

$$ \mathcal{U} (\bar{\beta}, \bar{\theta}, \bar{\phi})
= U (\vec{\beta}_{N_d}, \vec{\theta}_{N_d}, \vec{\phi}_{N_d}) \cdots
U (\vec{\beta}_1, \vec{\theta}_1, \vec{\phi}_1),
$$

where $\bar{\beta}, \bar{\theta}$, and $\bar{\phi}$ are matrices of dimensions $N_d \times 2$.
The matrix $\bar{\beta}$ is also complex-valued.

In [ ]:
def ecd_rot_ansatz(bmag_mat, barg_mat, theta_mat, phi_mat, nfocks):
    """
    ECD-rotation ansatz.

    Arguments:
    bmag_mat, barg_mat -- ECD parameters
    theta_mat, phi_mat -- rotation parameters
    nfocks -- Fock cutoffs
    """
    # Check
    if bmag_mat.shape != barg_mat.shape:
        raise ValueError("Dimensions of bmag_mat and barg_mat do not match.")
    beta_mat = get_cvec_np(bmag_mat, barg_mat)
    if beta_mat.shape != theta_mat.shape:
        raise ValueError("Lengths of beta_mat and theta_mat do not match.")
    if beta_mat.shape != phi_mat.shape:
        raise ValueError("Lengths of beta_mat and phi_mat do not match.")

    # Initialize
    ndepth = beta_mat.shape[0]
    uni = ecd_rot_op(beta_mat[0, :], theta_mat[0, :], phi_mat[0, :], nfocks)

    # Check
    if ndepth == 1:
        return uni

    # Loop through blocks
    for i in range(1, ndepth):
        new_uni = ecd_rot_op(beta_mat[i, :], theta_mat[i, :], phi_mat[i, :], nfocks)
        uni = ( new_uni * uni )

    return uni

## Hamiltonian

### Basics

In [ ]:
def decimal_to_binary(decimal_number, length):
    # Convert the decimal number to binary and strip the '0b' prefix
    binary_representation = bin(decimal_number)[2:]

    # Pad the binary representation with leading zeros if necessary
    padded_binary = binary_representation.zfill(length)

    # If the padded length is less than the specified length, raise an error
    if len(padded_binary) > length:
        raise ValueError("The binary representation is longer than the specified length.")

    return padded_binary


def binary_to_decimal(binary_string, length):
    # Validate that the input is a binary string of the specified length
    if len(binary_string) != length:
        raise ValueError(f"Input must be a binary string of length {length}.")

    if not all(bit in '01' for bit in binary_string):
        raise ValueError("Input must be a binary string.")

    # Convert the binary string to decimal
    decimal_number = int(binary_string, 2)

    return decimal_number

In [ ]:
def find_basis_state(state_vector):
    # Convert input to a Qobj if it isn't already
    if not isinstance(state_vector, qt.Qobj):
        state_vector = qt.Qobj(state_vector)

    # Get the number of qubits based on the length of the state vector
    N = int(np.log2(state_vector.shape[0]))

    # Check if the state vector is normalized
    if not np.isclose(state_vector.norm(), 1):
        raise ValueError("Input state vector must be normalized.")

    # Generate all possible basis states for N qubits
    basis_states = [qt.basis(2**N, i) for i in range(2**N)]

    # Check overlap with all basis states
    for index, b in enumerate(basis_states):
        overlap = state_vector.overlap(b)
        if np.isclose(overlap, 1):
            # Convert index to binary representation
            return format(index, f'0{N}b')  # Format as binary string with leading zeros

    # If no overlap found, return None (or raise an error)
    return None

### Qubit

In [ ]:
def binary_to_qubit_ham(H_bin, symbol_list, include_id=False):
    """
    Map a symbolic binary Hamiltonian to a spin Hamiltonian.

    Arguments:
    H_bin -- The binary Hamiltonian as a SymPy object
    symbol_list -- SymPy symbols defining the Hamiltonian
    include_id -- identity as symbol (True) or value 1 (False)
    """
    # Initialize spin variables (Z0, Z1, ..., Zn)
    z_symbols = sp.symbols('z:{}'.format(len(symbol_list)))

    # Define the identity operator (I_j)
    Ident = sp.symbols(r'\mathbb{I}') if include_id else 1.0

    # Create a mapping dictionary from binary symbols to spin expressions
    bin2spin_dict = {
        symbol: (1/2)*(Ident - z) for symbol, z in zip(symbol_list, z_symbols)
    }

    # Convert the binary Hamiltonian to a spin Hamiltonian
    spin_ham = H_bin.subs(bin2spin_dict).expand()

    # Z^2 = I
    sq_z = [z**2 for z in z_symbols]
    sq_values = [Ident] * len(z_symbols)  # All squared terms map to the identity
    spin_squared_map = dict(zip(sq_z, sq_values))

    # Substitute squared terms
    red_spin_ham = spin_ham.subs(spin_squared_map)

    return red_spin_ham


def check_spinz(input_list, spinz):
    out_val = ['I']*len(spinz)
    for ll in range(len(input_list)):
        out_val[int(input_list[ll].strip('z'))] = 'Z'
    return out_val


def sympy_to_pauli_dict(smpy_exp):
    """
    Convert a sympy spin Hamiltonian expression to a dictionary with
    Pauli words as keys and string coefficients as values.
    """
    # Determine the number of qubits
    spinz = smpy_exp.free_symbols

    # Split at spaces so we have the individual terms/coefficients
    split_expr = str(smpy_exp).split()

    # Firs iteration
    matrix_dict = {}
    split_term = split_expr[0].split('*')
    tmp_coeff = split_term[0]
    tmp_paulis = split_term[1:]
    pauli_word = ''.join(check_spinz(tmp_paulis, spinz))
    matrix_dict[pauli_word] = tmp_coeff

    # Iterate through the remaining terms
    for ii in range(1, len(split_expr), 2):
        tmp_sign = split_expr[ii]
        split_term = split_expr[ii+1].split('*')
        tmp_coeff  = split_term[0]
        tmp_paulis = split_term[1:]
        pauli_word = ''.join(check_spinz(tmp_paulis, spinz))
        matrix_dict[pauli_word] = tmp_sign+tmp_coeff

    return matrix_dict


def binary_to_pauli_list(H_total, symbol_list):
    """
    Maps a binary Hamiltonian to Pauli terms and coefficients.

    Arguments:
    H_total -- The binary Hamiltonian
    symbol_list -- symbols defining the Hamiltonian
    """
    spin_ham = binary_to_qubit_ham(H_total, symbol_list)
    op_dict = sympy_to_pauli_dict(spin_ham)

    return [[key, float(value)] for key, value in op_dict.items()]

### Qudit

In [ ]:
def matrices_to_qudit_list(matrices):
    """
    Maps list of matrices to qudit terms.

    Argument:
    matrices -- list of matrices
    """
    # Initialize the result list
    result = []

    # Get the number of matrices
    num_matrices = len(matrices)

    # Create a list to store diagonal values from each matrix
    diagonal_values = []

    # Collect diagonal values and their labels
    for index, matrix in enumerate(matrices):
        diag = np.diagonal(matrix)  # Get the diagonal elements
        diagonal_values.append((diag, index))  # Store as (diagonal_elements, index)

    # Recursive function to generate combinations of indices
    def generate_combinations(combination, depth):
        if depth == num_matrices:
            # Calculate the product for the current combination
            product = 1
            label = []
            for matrix_index, diag_index in combination:
                product *= matrices[matrix_index][diag_index, diag_index]
                label.append(f'P{diag_index}')
            result.append([ ', '.join(label), product ])
            return

        # Loop through the diagonal elements of the current matrix
        diag, matrix_index = diagonal_values[depth]
        for i in range(len(diag)):
            generate_combinations(combination + [(matrix_index, i)], depth + 1)

    # Start generating combinations
    generate_combinations([], 0)

    return result


def partition_string_list(input_list, partition_vector):
    """
    Partition the string in the input list based on the partition vector.

    Arguments:
    input_list -- A list where the first element is a string to partition,
                  and the last element is a value to retain.
    partition_vector -- A list of integers that specifies the lengths of the partitions.

    Returns:
    A new list with the partitioned string and the retained value.
    """
    # Ensure the partition_vector is valid
    string_part = input_list[0]
    total_length = sum(partition_vector)

    if total_length != len(string_part):
        raise ValueError("The sum of the partition vector must equal the length of the string.")

    # Partition the string based on the partition vector
    partitions = []
    start_index = 0

    for size in partition_vector:
        partitions.append(string_part[start_index:start_index + size])
        start_index += size

    # Return the modified list
    return partitions + [input_list[-1]]


def partitioned_pauli_term_to_qudit_term(pterm):
    mat_list = []
    for i in range(len(pterm) - 1):
        new_mat = generate_tensor_product(pterm[i])
        if i == 0:
            new_mat *= pterm[-1]
        mat_list.append(np.real( new_mat.full() ))

    return matrices_to_qudit_list(mat_list)


def pauli_list_to_qudit_terms(pterms, partition_vector):
    term_list = []
    for term in pterms:
        new_term = partition_string_list(term, partition_vector)
        qudit_term = partitioned_pauli_term_to_qudit_term(new_term)
        term_list.append(qudit_term)

    result_dict = {}
    for lst in term_list:
        for key, value in lst:
            if key in result_dict:
                result_dict[key] += value
            else:
                result_dict[key] = value
    return [[key, value] for key, value in result_dict.items()]

### Knapsack

In [ ]:
def binary_knapsack_ham(l_val, values, weights, max_weight, include_id=False):
    """
    Generates the binary Hamiltonian for the knapsack problem.

    Arguments:
    l_val -- lambda penalty parameter
    values -- item values
    weights -- item weights
    max_weight -- total weight capacity
    include_id -- identity as symbol (True) or value 1 (False)

    Returns:
    H_total -- The binary Hamiltonian
    """
    # Number of primary variables
    N_qb = len(weights)

    # Symbols
    symbol_list = list(sp.symbols('x_:{}'.format(str(N_qb))))
    Ident = sp.symbols(r'\mathbb{I}') if include_id else 1.

    # E = sum(i) v(i) x(i)
    H_prob = sum(values[ii] * symbol_list[ii] for ii in range(N_qb))

    # Calculate scaling factor
    max_weight_bin_str = bin(max_weight).lstrip('0b')  # Step 1
    max_val = 2**len(max_weight_bin_str) - 1  # Step 2
    scaling_factor = max_val / max_weight  # Step 3

    # Apply scaling to weights
    scaled_weights = [weight * scaling_factor for weight in weights]

    # W0 = sum(i) w(i) x(i)
    H_constraints = sum(scaled_weights[ii] * symbol_list[ii] for ii in range(N_qb))

    # Bitstring representation converted to list
    bin_weight = list(bin(max_weight).lstrip('0b'))[::-1]

    # Auxiliary symbol indices start after the primary variables
    aux_symbols = sp.symbols('x_{}:{}'.format(str(N_qb), str(N_qb + len(bin_weight))))

    # A = sum(i) 2^i y(i)
    H_constraints_aux = sum(aux_symbols[ii] * 2**ii for ii in range(len(bin_weight)))

    # Full Hamiltonian
    H_total = -H_prob + l_val * (max_weight - H_constraints - H_constraints_aux)**2

    # Construct full list of symbols in expression
    symbol_list.extend(list(aux_symbols))

    # Create a list of x_j**2
    sq_syms = [temp_sym**2 for temp_sym in symbol_list]

    # Maps x_j^{2} to x_j:
    conv_dict = dict(zip(sq_syms, list(symbol_list)))

    # Final binary Hamiltonian
    H_total = H_total.subs(conv_dict)

    return H_total, symbol_list

In [ ]:
def generate_tensor_product(string):
    """
    Get QuTip object given a string representing a Pauli word.
    """
    # Define a mapping of characters to corresponding QuTiP operators
    operator_map = {
        'I': qt.qeye(2),  # Identity operator
        'X': qt.sigmax(),  # Pauli-X operator
        'Y': qt.sigmay(),  # Pauli-Y operator
        'Z': qt.sigmaz()   # Pauli-Z operator
    }

    # Create a list to collect the operators
    operators = []

    # Append the corresponding operators based on the input string
    for char in string:
        operators.append(operator_map[char])

    # Compute the tensor product of all operators in the list
    U = qt.tensor(*operators).full()

    return qt.Qobj(U)


def qubit_op_to_ham(pterms):
    """
    Get QuTip object given a set of Pauli words and correspdoing coefficients.
    """
    terms = []
    for p in pterms:
        term = ( p[1] * generate_tensor_product(p[0]) )
        terms.append(term)

    return sum(terms)

## Loss function

We want to minimize the following cost function:

$$ \min_{ \overrightarrow{\beta}, \overrightarrow{\theta}, \overrightarrow{\phi} } E
= \langle \psi (\overrightarrow{\beta}, \overrightarrow{\theta}, \overrightarrow{\phi}) | \:
H \: |\psi (\overrightarrow{\beta}, \overrightarrow{\theta}, \overrightarrow{\phi})\rangle,
$$
where $H$ is the two-qumode Hamiltonian and $|\psi \rangle$ is the two-qumode state traced from the ECD-rotation ansatz $|\Psi \rangle$.

In [ ]:
def state_from_ecd(Xvec, ndepth, nfocks):
    """
    Qumode state |Psi> = U ( |0> |0, 0> ).

    Arguments:
    Xvec -- ECD-rotation parameters
    ndepth -- circuit depth
    nfocks -- Fock cutoffs
    """
    # Parameters
    beta_mag, beta_arg, theta, phi = unpack_variables(Xvec, ndepth)

    # ECD unitary
    U = ecd_rot_ansatz(beta_mag, beta_arg, theta, phi, nfocks)

    # U |0, 0, 0>
    vac = qt.tensor( qt.basis(2, 0), qt.basis(nfocks[0], 0), qt.basis(nfocks[1], 0) )
    psi = U * vac

    return psi

In [ ]:
def energy_val(Xvec, ndepth, nfocks, H):
    """
    Compute <psi | H |psi > where

    |psi (n, m)> <== U ( |0> |0, 0> ).

    Arguments:
    Xvec -- ansatz ECD-rotation parameters
    H -- QuTip two-qumode Hamiltonian
    nfock -- Fock cutoff for qumode
    ndepth -- circuit depth
    """
    # Qubit-qubit-qumode state
    psi = state_from_ecd(Xvec, ndepth, nfocks)
    psi = qt.Qobj( psi.full() )

    # Expectation value
    ham = qt.Qobj( H.full() )
    en = qt.expect(ham, psi)

    return en

## Analysis

In [ ]:
def generate_triples(nfocks):
    # Create ranges for q, n, and m
    q_range = np.arange(2)
    n_range = np.arange(nfocks[0])
    m_range = np.arange(nfocks[1])

    # Create a meshgrid of q, n, and m with valid indexing
    q_grid, n_grid, m_grid = np.meshgrid(q_range, n_range, m_range, indexing='ij')

    # Stack the grids to get (q, n, m) triples
    triples = np.stack((q_grid.ravel(), n_grid.ravel(), m_grid.ravel()), axis=-1)

    return triples

In [ ]:
def num_prob_basis(Xvec, nvec, ndepth, nfocks):
    """
    | <psi | q, n, m> |^2, where

    |psi> <== U |0, 0, 0>.

    Arguments:
    Xvec -- ansatz parameters
    nvec -- Fock basis state indices
    nfocks -- Fock cutoffs
    ndepth -- circuit depth
    """
    # Qubit-qubit-qumode state
    psi = state_from_ecd(Xvec, ndepth, nfocks)

    # |q, n, m >
    state = qt.tensor(qt.basis(2, nvec[0]),
                      qt.basis(nfocks[0], nvec[1]),
                      qt.basis(nfocks[1], nvec[2]) )

    # Expectation value
    P0 = psi.overlap(state)

    return np.abs(P0)**2


def num_prob_all(Xvec, ndepth, nfocks):
    """
    | <psi | q, n, m> |^2 for all (n, m).

    Arguments:
    Xvec -- ansatz parameters
    nfock -- Fock cutoff for single qumode
    nvec -- Fock basis state indices
    """
    # Initialize
    N1 = generate_triples(nfocks)
    ntriples = N1.shape[0]

    # Generate
    P1 = []
    for i in range(ntriples):
        P1.append( num_prob_basis(Xvec, N1[i, :], ndepth, nfocks) )

    return np.array(P1)

## Optimization

In [ ]:
def ecd_opt_vqe(H, ndepth, nfocks, maxiter=100, method='COBYLA', verb=0,
                threshold=1e-08, print_freq=10, Xvec=[]):
    """
    Minimize the cost function using SciPy-based methods.

    Arguments:
    H -- QuTip Hamiltonian
    ndepth -- ansatz circuit depth
    nfocks -- Fock cutoffs
    maxiter -- maximum number of iterations
    method -- optimization method
    threshold -- error tolerance
    Xvec -- optional initial guesses
    print_freq -- frequency of printing and storing intermediate results
    """
    # Bound parameters
    beta_mag_min = 0.0
    beta_mag_max = 10.0
    beta_arg_min = 0.0
    beta_arg_max = 2 * np.pi
    theta_min = 0.0
    theta_max = np.pi
    phi_min = 0.0
    phi_max = 2 * np.pi

    # Define bounds
    size = ndepth * 2
    beta_mag_bounds = [(beta_mag_min, beta_mag_max)] * size
    beta_arg_bounds = [(beta_arg_min, beta_arg_max)] * size
    theta_bounds = [(theta_min, theta_max)] * size
    phi_bounds = [(phi_min, phi_max)] * size
    bounds = beta_mag_bounds + beta_arg_bounds + theta_bounds + phi_bounds

    # Guess
    if len(Xvec) == 0:
        beta_mag = np.random.uniform(0, 3, size=(ndepth, 2))
        beta_arg = np.random.uniform(0, np.pi, size=(ndepth, 2))
        theta = np.random.uniform(0, np.pi, size=(ndepth, 2))
        phi = np.random.uniform(0, np.pi, size=(ndepth, 2))
        Xvec = pack_variables(beta_mag, beta_arg, theta, phi)

    # Loss function
    obj_fun = partial(energy_val, ndepth=ndepth, nfocks=nfocks, H=H)

    # Intermediate values
    iteration_step = 0
    intermediate_results = []

    def callback(xk):
        nonlocal iteration_step
        iteration_step += 1
        loss_value = obj_fun(xk)
        if verb == 1 and (iteration_step % print_freq == 0):
            print("-------------------")
            print(f"iter: {iteration_step}")
            print(f"fval: {loss_value}")

        # Store intermediate results
        if iteration_step % print_freq == 0:
            intermediate_results.append((loss_value, xk.copy()))

    # SciPy options
    options = {'disp': True, 'maxiter': maxiter}

    # Optimize
    result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds,
                             tol=threshold, options=options, callback=callback)

    return result.fun, result.x, intermediate_results

## Explore

Let us discuss the binary knapsack problem of $N$ items

\begin{align*}
\max_{\mathbf{x}} E
&= \sum_{j = 1}^N v_j \: x_j,
\\
\text{subject to} \quad W_0 (\mathbf{x})
&= \sum_{j = 1}^N w_j \: x_j \leq W,
\end{align*}
where $\{ v_j \}$ are values, $\{ w_j \}$ are weights, and $W$ is the total weight capacity.
The cost function optimization becomes

$$ \min_{\mathbf{x}, \mathbf{y}} F
= - E (\mathbf{x})
+ \lambda \: \Big[ W - W_0 (\mathbf{x})
- \sum_{j = 1}^M \: 2^{j - 1} \: y_j \Big]^2,
$$
where the sum of $M = \lceil \log_2 (W + 1) \rceil$ terms represent the auxiliary variables with $\lambda$ being the quadratic penalty coefficient.

In [ ]:
values = [2, 5, 7, 3]
weights = [2.5, 3, 4, 3.5]
max_weight = 7
l_val = 2

bkp_fun1, bkp_list1 = binary_knapsack_ham(l_val, values, weights, max_weight)
bkp_fun1

-2*x_0 - 5*x_1 - 7*x_2 - 3*x_3 + 2*(-2.5*x_0 - 3.0*x_1 - 4.0*x_2 - 3.5*x_3 - x_4 - 2*x_5 - 4*x_6 + 7)**2

Qubit Hamiltonian.

In [ ]:
bkp_list1 = binary_to_pauli_list(bkp_fun1, bkp_list1)
bkp_list1

[['ZZIIIII', 7.5],
 ['ZIZIIII', 10.0],
 ['ZIIZIII', 8.75],
 ['ZIIIZII', 2.5],
 ['ZIIIIZI', 5.0],
 ['ZIIIIIZ', 10.0],
 ['ZIIIIII', -14.0],
 ['IZZIIII', 12.0],
 ['IZIZIII', 10.5],
 ['IZIIZII', 3.0],
 ['IZIIIZI', 6.0],
 ['IZIIIIZ', 12.0],
 ['IZIIIII', -15.5],
 ['IIZZIII', 14.0],
 ['IIZIZII', 4.0],
 ['IIZIIZI', 8.0],
 ['IIZIIIZ', 16.0],
 ['IIZIIII', -20.5],
 ['IIIZZII', 3.5],
 ['IIIZIZI', 7.0],
 ['IIIZIIZ', 14.0],
 ['IIIZIII', -19.5],
 ['IIIIZZI', 2.0],
 ['IIIIZIZ', 4.0],
 ['IIIIZII', -6.0],
 ['IIIIIZZ', 8.0],
 ['IIIIIZI', -12.0],
 ['IIIIIIZ', -24.0],
 ['IIIIIII', 41.75]]

In [ ]:
bkp_ham1 = qt.Qobj( qubit_op_to_ham(bkp_list1).full() )
evals, evecs = bkp_ham1.eigenstates()

In [ ]:
evals[:10]

array([-12. , -10. ,  -9.5,  -8.5,  -8.5,  -7.5,  -7.5,  -7. ,  -6.5,
        -6.5])

In [ ]:
for i in range(10):
    print(f"State {i}: ", find_basis_state(evecs[i]))

State 0:  0110000
State 1:  0110100
State 2:  0011000
State 3:  1010000
State 4:  1010100
State 5:  0101000
State 6:  0101100
State 7:  0010110
State 8:  1100010
State 9:  1100100


Fock Hamiltonian.

In [ ]:
bkp_part1 = [1, 3, 3,] # Qubit partition

bkp_ham2 = pauli_list_to_qudit_terms(bkp_list1, bkp_part1)
bkp_ham2

[['P0, P0, P0', 98.0],
 ['P0, P0, P1', 18.0],
 ['P0, P0, P2', 50.0],
 ['P0, P0, P3', 2.0],
 ['P0, P0, P4', 72.0],
 ['P0, P0, P5', 8.0],
 ['P0, P0, P6', 32.0],
 ['P0, P0, P7', 0.0],
 ['P0, P1, P0', 21.5],
 ['P0, P1, P1', -2.5],
 ['P0, P1, P2', 1.5],
 ['P0, P1, P3', 9.5],
 ['P0, P1, P4', 9.5],
 ['P0, P1, P5', 1.5],
 ['P0, P1, P6', -2.5],
 ['P0, P1, P7', 21.5],
 ['P0, P2, P0', 11.0],
 ['P0, P2, P1', -5.0],
 ['P0, P2, P2', -5.0],
 ['P0, P2, P3', 11.0],
 ['P0, P2, P4', 1.0],
 ['P0, P2, P5', 1.0],
 ['P0, P2, P6', -7.0],
 ['P0, P2, P7', 25.0],
 ['P0, P3, P0', -9.5],
 ['P0, P3, P1', 30.5],
 ['P0, P3, P2', 2.5],
 ['P0, P3, P3', 74.5],
 ['P0, P3, P4', -5.5],
 ['P0, P3, P5', 50.5],
 ['P0, P3, P6', 14.5],
 ['P0, P3, P7', 102.5],
 ['P0, P4, P0', 27.0],
 ['P0, P4, P1', -5.0],
 ['P0, P4, P2', 3.0],
 ['P0, P4, P3', 3.0],
 ['P0, P4, P4', 13.0],
 ['P0, P4, P5', -3.0],
 ['P0, P4, P6', -3.0],
 ['P0, P4, P7', 13.0],
 ['P0, P5, P0', -7.5],
 ['P0, P5, P1', 16.5],
 ['P0, P5, P2', -3.5],
 ['P0, P5, P3', 52.5],

Optimization.

In [ ]:
nfocks = [8, 8]
ndepth = 5

In [ ]:
en, Xvec, int_results = ecd_opt_vqe(bkp_ham1, ndepth, nfocks, maxiter=200, method='BFGS',
                                    verb=1, threshold=1e-12)
en

<ipython-input-21-546bbbcad45f>:66: RuntimeWarning: Method BFGS cannot handle bounds.
  result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds,


-------------------
iter: 10
fval: 1.0144571586242686
-------------------
iter: 20
fval: -6.334910912495342
-------------------
iter: 30
fval: -8.278028409573478
-------------------
iter: 40
fval: -9.478233339449334
-------------------
iter: 50
fval: -10.327837376245908
-------------------
iter: 60
fval: -11.509175156521225
-------------------
iter: 70
fval: -11.821581539851751
-------------------
iter: 80
fval: -11.9588615689411
-------------------
iter: 90
fval: -11.991235718481052
-------------------
iter: 100
fval: -11.994098175878396
-------------------
iter: 110
fval: -11.995047161485346
-------------------
iter: 120
fval: -11.995656508736353
-------------------
iter: 130
fval: -11.995863291089687
-------------------
iter: 140
fval: -11.995963422404083
-------------------
iter: 150
fval: -11.995995999444064
-------------------
iter: 160
fval: -11.996026208426889
-------------------
iter: 170
fval: -11.996041852348526
-------------------
iter: 180
fval: -11.99605322592974
--------

-11.996059486626715

In [ ]:
int_results

[(1.0144571586242686,
  array([ 2.48739857, -0.12819412,  0.88434011,  0.43764229,  1.67086808,
          0.49781909,  0.57462852, -0.62726047,  2.57705518,  0.1944916 ,
          1.14474476,  0.69802012,  0.92246096,  2.84541755,  0.99919422,
          1.48834651,  0.30989539,  3.19382233,  2.17985267,  0.21961605,
          2.55260936,  3.06824862,  2.21747252,  1.98349449,  3.09151926,
          1.51403241,  0.86229736,  2.01097929,  1.61897112,  2.9558595 ,
          0.39720226,  2.97984452,  1.41282515,  2.61228042,  3.8082838 ,
          2.81306479,  1.48460125,  2.17267392,  0.84214299,  2.16563646])),
 (-6.334910912495342,
  array([ 1.90284439,  0.09920202,  1.10510627, -0.09652391,  1.66792679,
          0.08532734,  0.94868686, -0.11853647,  3.29192597, -0.11900798,
          1.16738285,  1.14039173,  1.01012179,  2.6459994 ,  1.18880303,
          1.19089898,  0.13692227,  3.28409532,  2.05291243,  0.18383694,
          2.94312169,  3.01135765,  3.23508406,  1.8153325 ,  3.7